
<h1 align="center">Дескрипторы</h1>

Дескрипторы - это способ описания молекул в виде набора числовых характеристик, пригодного для машинного обучения.

**Виды дескрипторов, доступных для вычисления, согласно функциональной классификации:**
<ul><li>Конституционные</li>
<li>Топологические</li>
<li>Поверхности</li>
<li>Фрагментные</li>
<li>Фармакофорные</li>
<li>Физикохимические</li>
</ul>

## RDkit

В RDKit реализованы многие из них. Например:
<ul><li>
HeavyAtomCount
</li>
<li>MACCS keys
</li>
<li>TPSA
</li>
<li>Morgan/Circular Fingerprints
</li>
<li>2D Pharmacophore Fingerprints
</li>
</ul>
Список доступных для вычисления дескрипторов для библиотеки RDKit доступен по ссылке:   http://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors

В качестве источника данных для данного тюториала выступает файл data/logBB.sdf, содержащий соединения, для которых известен LogBB.  
Болезнь Паркинсона (БП) - это прогрессирующее нейродегенеративное заболевание, поражающее 2% населения в возрасте старше 60 лет. На сегодняшний день не существует лекарств, модифицирующих заболевание, для предотвращения потери дофаминергических нейронов и ненормального отложения белка в мозге. Существует большая потребность в нейропротекторной терапии для предотвращения или замедления дофаминергической дегенерации нейронов. Важной предпосылкой для соединения, предназначенного для воздействия на центральную нервную систему (ЦНС), является удовлетворительный транспорт через гематоэнцефалический барьер (ГЭБ). LogBB является показателем проницаемости ГЭБ.

In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.preprocessing import FunctionTransformer
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import rdkit.Chem.AllChem as AllChem
from numpy import zeros
from rdkit import DataStructs
import pandas as pd

Подготовим молекулы для работы:

In [2]:
# читаем из файла
molecules = [mol for mol in Chem.SDMolSupplier("data/logBB.sdf") if mol is not None]
print(f'Количество молекул = {len(molecules)}')

[23:09:55] Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17 18 19 20 21
[23:09:55] ERROR: Could not sanitize molecule ending on line 3192
[23:09:55] ERROR: Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17 18 19 20 21
[23:09:55] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 17 18
[23:09:55] ERROR: Could not sanitize molecule ending on line 4541
[23:09:55] ERROR: Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 17 18
[23:09:55] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 12 13 14 15 16
[23:09:55] ERROR: Could not sanitize molecule ending on line 5314
[23:09:55] ERROR: Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 12 13 14 15 16
[23:09:55] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 14 15 16 17 18
[23:09:55] ERROR: Could not sanitize molecule ending on line 7252
[23:09:55] ERROR: Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 14 15 16 17 18
[23:09:55] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 11 12 13 14 15
[23:09:55] ERROR

Количество молекул = 299


Воспользуемся возможностями библиотеки Scikit-learn (Sklearn) для создания цельного
генератора дескрипторов.

In [3]:

# создаем словарь из дескриторов структуры
ConstDescriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
                    "NHOHCount": Descriptors.NHOHCount,
                    "NOCount": Descriptors.NOCount,
                    "NumHAcceptors": Descriptors.NumHAcceptors,
                    "NumHDonors": Descriptors.NumHDonors,
                    "NumHeteroatoms": Descriptors.NumHeteroatoms,
                    "NumRotatableBonds": Descriptors.NumRotatableBonds,
                    "NumValenceElectrons": Descriptors.NumValenceElectrons,
                    "NumAromaticRings": Descriptors.NumAromaticRings,
                    "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
                    "RingCount": Descriptors.RingCount}

# создаем словарь из физико-химических дескрипторов                            
PhisChemDescriptors = {"MW": Descriptors.MolWt,
                       "LogP": Descriptors.MolLogP,
                       "MR": Descriptors.MolMR,
                       "TPSA": Descriptors.TPSA}

# объединяем все дескрипторы в один словарь
descriptors = {}
descriptors.update(ConstDescriptors)
descriptors.update(PhisChemDescriptors)
print(f"Количество дескрипторов в словаре: {len(descriptors)}")


# функция для генерации дескрипторов из молекул
def mol_dsc_calc(mols): 
    return DataFrame({k: f(m) for k, f in descriptors.items()} 
             for m in mols)

# оформляем sklearn трансформер для использования в конвеерном моделировании (sklearn Pipeline)
descriptors_transformer = FunctionTransformer(mol_dsc_calc, validate=False)

Количество дескрипторов в словаре: 15


`descriptors_transformer` – это специальный объект, который может быть встроен в конвейер моделирования. Генерация дескрипторов этим объектом показана ниже.

In [4]:
X = descriptors_transformer.transform(molecules)

In [5]:
X

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,5,0,0,0,0,3,0,32,0,0,0,133.405,2.3765,25.9640,0.00
1,5,0,0,0,0,3,1,32,0,0,0,133.405,2.0289,26.2140,0.00
2,5,0,0,0,0,3,0,30,0,0,0,98.479,1.9631,16.1520,0.00
3,5,1,1,1,1,1,2,32,0,0,0,74.123,0.7788,21.9938,20.23
4,5,0,1,1,0,1,1,30,0,0,0,72.107,0.9854,20.9720,17.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,34,2,4,3,2,4,1,190,0,0,5,470.694,6.4126,133.0716,74.60
295,35,4,9,7,3,10,11,192,2,1,3,505.637,2.4028,133.2327,131.19
296,36,6,15,15,5,19,9,192,2,2,4,575.632,-2.3442,127.8725,223.09
297,39,7,12,12,6,12,5,208,2,1,5,543.525,0.0013,131.7544,206.07


Выше показана часть матрицы дескрипторов для моделируемого набора данных.

## Нормализация дескрипторов

Самые используемые в хемоинформатике методы предобработки дескрипторов:
<ol><li> Центрирование данных:  
    \begin{equation}x_i'=x_i-\bar x\end{equation} </li>
<li> Нормировка (стандартизация) - приводит каждую из числовых характеристик к нулевому среднему и единичному стандартному отклонению:  
<br/>\begin{equation}z_i = \frac{x_i-\bar x}{s},\end{equation}
<br/>где s - стандартное отклонение:  
<br/>\begin{equation}s = \sqrt{\frac{1}{n - 1}\sum \limits_{i=1}^{n}(x_i - \bar x)^2}\end{equation}
</li>  
<li>Линейное преобразование, переводящее минимальное по выборке значение шкалируемого параметра к нижней границе (например, к 0), а максимальное к верхней (например, к 1):  
<br/> \begin{equation} x' = \frac{x_i - min(x)}{max(x) - min(x)}\end{equation}
</li>
</ol>

#### Пример использования

Попробуем нормализовать ранее вычисленные дескрипторы, приведя каждую из числовых характеристик к нулевому среднему и единичному стандартному отклонению:

`1.` Используя особенности типа данных pandas DataFrame:

In [6]:
# Применим напрямую вышеприведенную формулу
# X.std() - позволяет вычислить стандартное отклонение для каждого столбца дескрипторов
# X.mean() - позволяет вычислить среднее для каждого столбца дескрипторов
# В результате X_norm_df будет содержать уже нормализованные дескрипторы
X_norm_df = (X - X.mean())/ X.std() 
X_norm_df

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,-1.479893,-0.836903,-1.344831,-1.354238,-0.867749,-0.360115,-1.138554,-1.428676,-1.255087,-0.817600,-1.357038,-0.991762,-0.098418,-1.323091,-1.193554
1,-1.479893,-0.836903,-1.344831,-1.354238,-0.867749,-0.360115,-0.785779,-1.428676,-1.255087,-0.817600,-1.357038,-0.991762,-0.327275,-1.315325,-1.193554
2,-1.479893,-0.836903,-1.344831,-1.354238,-0.867749,-0.360115,-1.138554,-1.475844,-1.255087,-0.817600,-1.357038,-1.291897,-0.370597,-1.627902,-1.193554
3,-1.479893,-0.057358,-0.924659,-0.872767,0.052311,-1.113084,-0.433004,-1.428676,-1.255087,-0.817600,-1.357038,-1.501199,-1.150332,-1.446425,-0.606585
4,-1.479893,-0.836903,-0.924659,-0.872767,-0.867749,-1.113084,-0.785779,-1.475844,-1.255087,-0.817600,-1.357038,-1.518524,-1.014308,-1.478168,-0.698271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,2.025202,0.722187,0.335857,0.090175,0.972371,0.016369,-0.785779,2.297571,-1.255087,-0.817600,1.968818,1.906714,2.558919,2.004218,0.970949
295,2.146067,2.281277,2.436716,2.016059,1.892431,2.275275,2.741969,2.344738,0.704553,0.563543,0.638475,2.206995,-0.081102,2.009223,2.612896
296,2.266933,3.840367,4.957748,5.867827,3.732551,5.663633,2.036419,2.344738,0.704553,1.944687,1.303646,2.808494,-3.206491,1.842707,5.279354
297,2.629529,4.619912,3.697232,4.423414,4.652611,3.028243,0.625320,2.722080,0.704553,0.563543,1.968818,2.532584,-1.662232,1.963299,4.785523


`2.` Используя StandardScaler из библиотеки scikit-learn:

In [7]:
scaler = StandardScaler()
scaler.fit(X.values)
X_norm_SS = DataFrame(scaler.transform(X.values), index=X.index, columns=X.columns)
X_norm_SS

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,-1.482374,-0.838306,-1.347086,-1.356508,-0.869204,-0.360719,-1.140463,-1.431071,-1.257191,-0.818971,-1.359313,-0.993425,-0.098583,-1.325309,-1.195555
1,-1.482374,-0.838306,-1.347086,-1.356508,-0.869204,-0.360719,-0.787097,-1.431071,-1.257191,-0.818971,-1.359313,-0.993425,-0.327824,-1.317530,-1.195555
2,-1.482374,-0.838306,-1.347086,-1.356508,-0.869204,-0.360719,-1.140463,-1.478318,-1.257191,-0.818971,-1.359313,-1.294063,-0.371219,-1.630631,-1.195555
3,-1.482374,-0.057454,-0.926210,-0.874230,0.052399,-1.114950,-0.433730,-1.431071,-1.257191,-0.818971,-1.359313,-1.503716,-1.152260,-1.448850,-0.607602
4,-1.482374,-0.838306,-0.926210,-0.874230,-0.869204,-1.114950,-0.787097,-1.478318,-1.257191,-0.818971,-1.359313,-1.521069,-1.016008,-1.480646,-0.699442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,2.028597,0.723398,0.336420,0.090326,0.974001,0.016396,-0.787097,2.301423,-1.257191,-0.818971,1.972118,1.909911,2.563209,2.007578,0.972577
295,2.149665,2.285102,2.440801,2.019439,1.895604,2.279089,2.746565,2.348669,0.705734,0.564488,0.639546,2.210695,-0.081238,2.012591,2.617276
296,2.270733,3.846805,4.966060,5.877664,3.738809,5.673128,2.039833,2.348669,0.705734,1.947947,1.305832,2.813202,-3.211866,1.845797,5.288205
297,2.633937,4.627657,3.703430,4.430830,4.660411,3.033320,0.626368,2.726643,0.705734,0.564488,1.972118,2.536830,-1.665018,1.966590,4.793545


`3.` Используя особенности типа данных numpy array:

In [8]:
n = X.values.shape[1]
data = X.values
for i in range(n):
    vals = data[:,i].astype("float")
    m = vals.mean()
    s = vals.std()
    data[:,i] = (vals - m)/s
X_norm_np = pd.DataFrame(data, columns=X.columns)
X_norm_np

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,-1.482374,-0.838306,-1.347086,-1.356508,-0.869204,-0.360719,-1.140463,-1.431071,-1.257191,-0.818971,-1.359313,-0.993425,-0.098583,-1.325309,-1.195555
1,-1.482374,-0.838306,-1.347086,-1.356508,-0.869204,-0.360719,-0.787097,-1.431071,-1.257191,-0.818971,-1.359313,-0.993425,-0.327824,-1.317530,-1.195555
2,-1.482374,-0.838306,-1.347086,-1.356508,-0.869204,-0.360719,-1.140463,-1.478318,-1.257191,-0.818971,-1.359313,-1.294063,-0.371219,-1.630631,-1.195555
3,-1.482374,-0.057454,-0.926210,-0.874230,0.052399,-1.114950,-0.433730,-1.431071,-1.257191,-0.818971,-1.359313,-1.503716,-1.152260,-1.448850,-0.607602
4,-1.482374,-0.838306,-0.926210,-0.874230,-0.869204,-1.114950,-0.787097,-1.478318,-1.257191,-0.818971,-1.359313,-1.521069,-1.016008,-1.480646,-0.699442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,2.028597,0.723398,0.336420,0.090326,0.974001,0.016396,-0.787097,2.301423,-1.257191,-0.818971,1.972118,1.909911,2.563209,2.007578,0.972577
295,2.149665,2.285102,2.440801,2.019439,1.895604,2.279089,2.746565,2.348669,0.705734,0.564488,0.639546,2.210695,-0.081238,2.012591,2.617276
296,2.270733,3.846805,4.966060,5.877664,3.738809,5.673128,2.039833,2.348669,0.705734,1.947947,1.305832,2.813202,-3.211866,1.845797,5.288205
297,2.633937,4.627657,3.703430,4.430830,4.660411,3.033320,0.626368,2.726643,0.705734,0.564488,1.972118,2.536830,-1.665018,1.966590,4.793545


------

# MinMax

In [9]:
# ручная стандартизация
df_scaled_manual = (X - X.min()) / (X.max() - X.min())
df_scaled_manual

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,0.043478,0.000000,0.000000,0.000000,0.000000,0.157895,0.000000,0.057377,0.00,0.000000,0.0,0.145241,0.569719,0.085375,0.000000
1,0.043478,0.000000,0.000000,0.000000,0.000000,0.157895,0.076923,0.057377,0.00,0.000000,0.0,0.145241,0.532964,0.086784,0.000000
2,0.043478,0.000000,0.000000,0.000000,0.000000,0.157895,0.000000,0.049180,0.00,0.000000,0.0,0.089695,0.526006,0.030063,0.000000
3,0.043478,0.142857,0.066667,0.066667,0.166667,0.052632,0.153846,0.057377,0.00,0.000000,0.0,0.050959,0.400780,0.062994,0.090681
4,0.043478,0.000000,0.066667,0.066667,0.000000,0.052632,0.076923,0.049180,0.00,0.000000,0.0,0.047753,0.422626,0.057234,0.076516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0.673913,0.285714,0.266667,0.200000,0.333333,0.210526,0.076923,0.704918,0.00,0.000000,1.0,0.681665,0.996489,0.689155,0.334394
295,0.695652,0.571429,0.600000,0.466667,0.500000,0.526316,0.846154,0.713115,0.50,0.333333,0.6,0.737238,0.572500,0.690064,0.588059
296,0.717391,0.857143,1.000000,1.000000,0.833333,1.000000,0.692308,0.713115,0.50,0.666667,0.8,0.848558,0.070559,0.659847,1.000000
297,0.782609,1.000000,0.800000,0.800000,1.000000,0.631579,0.384615,0.778689,0.50,0.333333,1.0,0.797495,0.318569,0.681730,0.923708


In [10]:
# стандартизация через MinMaxScaler
scaler = MinMaxScaler()
df_scaled_sklearn = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
df_scaled_sklearn

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,0.043478,0.000000,0.000000,0.000000,0.000000,0.157895,0.000000,0.057377,0.00,0.000000,0.0,0.145241,0.569719,0.085375,0.000000
1,0.043478,0.000000,0.000000,0.000000,0.000000,0.157895,0.076923,0.057377,0.00,0.000000,0.0,0.145241,0.532964,0.086784,0.000000
2,0.043478,0.000000,0.000000,0.000000,0.000000,0.157895,0.000000,0.049180,0.00,0.000000,0.0,0.089695,0.526006,0.030063,0.000000
3,0.043478,0.142857,0.066667,0.066667,0.166667,0.052632,0.153846,0.057377,0.00,0.000000,0.0,0.050959,0.400780,0.062994,0.090681
4,0.043478,0.000000,0.066667,0.066667,0.000000,0.052632,0.076923,0.049180,0.00,0.000000,0.0,0.047753,0.422626,0.057234,0.076516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0.673913,0.285714,0.266667,0.200000,0.333333,0.210526,0.076923,0.704918,0.00,0.000000,1.0,0.681665,0.996489,0.689155,0.334394
295,0.695652,0.571429,0.600000,0.466667,0.500000,0.526316,0.846154,0.713115,0.50,0.333333,0.6,0.737238,0.572500,0.690064,0.588059
296,0.717391,0.857143,1.000000,1.000000,0.833333,1.000000,0.692308,0.713115,0.50,0.666667,0.8,0.848558,0.070559,0.659847,1.000000
297,0.782609,1.000000,0.800000,0.800000,1.000000,0.631579,0.384615,0.778689,0.50,0.333333,1.0,0.797495,0.318569,0.681730,0.923708


## Генерация молекулярных отпечатков при помощи библиотеки RDkit

Молекулярные отпечатки – еще один тип дескрипторов, показывающий хорошие результаты моделирования. Данные дескрипторы не имеет смысла объединять с ранее описанными.

In [11]:
def calc_morgan(mols):
    """ генерация молекулярных отпечатков по методу Моргана с радиусом 2
    """
    for_df = []
    for m in mols:
        arr = zeros((1,), dtype=int)
        DataStructs.ConvertToNumpyArray(AllChem.GetMorganFingerprintAsBitVect(m, 2), arr)
        for_df.append(arr)
    return DataFrame(for_df)

In [12]:
morgan_transformer = FunctionTransformer(calc_morgan, validate=False)

X = morgan_transformer.transform(molecules)

[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerator
[23:09:57] DEPRECATION WARNING: please use MorganGenerat

In [13]:
X

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
297,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Вопросы  
1. Сравните X_norm_df и X_norm_SS. Почему значения нормализованных дескрипторов не совпадают?
2. Чем отличаются "оценка стандартного отклонения на основании несмещённой оценки дисперсии" и "оценка стандартного отклонения на основании смещённой оценки дисперсии"?
3. Почему важно сохранять значения среднего и стандартного отклонения для каждого столбца дескрипторов?

----

# Ответы

1. Ответы не совпадают из-за того, что StandardScaler() по умолчанию использует несмещенную оценку стандартного отклонения (делит на n − 1, где n — количество наблюдений). Это стандартное поведение многих статистических библиотек. Чтобы результат совпадал : `X_norm_df = (X - X.mean()) / X.std(ddof=0)`

2. Несмещенная оценка дисперсии вычисляется по следующей формуле: 

<img src='img/ss.png'>

Деление на n−1 вместо n компенсирует "смещение", возникающее из-за использования выборочного среднего (μ) генеральной совокупности.

Смещенная оценка дисперсии:

<img src='img/df.png'>

Разница возникает из-за разных знаменетелях. Деление на n даёт "смещённую" оценку, так как она систематически занижает дисперсию для малых выборок.

А стандартное отклоение - это корень из соответсвующей дисперсии

<img src='img/s.png'>


_Несмещённая оценка используется:_

* В статистике для корректной оценки параметров генеральной совокупности.
* В выборках малого размера.

_Смещённая оценка используется :_

* В машинном обучении, когда мы работаем с большими выборками и смещение становится незначительным.
* Для внутреннего анализа выборки без необходимости оценивать параметры генеральной совокупности.


3. В случае многомерных данных, у каждого столбца могут быть свои значения 𝜇 и 𝜎. Сохранение этих значений для каждого столбца обеспечивает согласованную нормализацию/стандартизацию в будущих вычислениях.
